# NPL
Utilise Word2Vec, LSTM et ce dataset :  SMS Spam Collection Dataset = https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset?resource=download

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import random as r
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import recall_score, accuracy_score, fbeta_score, roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay

## Data

In [ ]:
data = pd.read_csv('spam.csv', encoding="ISO-8859-1")
data.head()

### Data Preparation

In [ ]:
# Télécharger la liste des stop words
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

# Fonction de nettoyage de texte
def clean_text(text):
    # Retirer les caractères non-alphabétiques et convertir en minuscules
    tokens = word_tokenize(text.lower())
    # Retirer les stop words
    stopped_tokens = [w for w in tokens if w not in stopwords.words('english')]
    return ' '.join(stopped_tokens)

# Appliquer la fonction de nettoyage à chaque message
data['v2'] = data['v2'].apply(clean_text)
data.head()
print("Word count: ", data['v2'].apply(lambda x: len(x.split(' '))).sum())
print("Word Token: ", data['v2'].apply(lambda x: len(word_tokenize(x))).sum())

### Entrainement de Word2Vec

In [ ]:

# Préparer les données pour Word2Vec (liste de listes de mots)
sentences = [message.split() for message in data['v2']]

# Entraîner un modèle Word2Vec
wordvec = Word2Vec(sentences, vector_size=50, window=4, min_count=1, workers=3, epochs=100)

# Fonction pour vectoriser un message en utilisant les vecteurs de ses mots
def vectorize_message(message):
    words = message.split()  # Tokenisation simple
    # On filtre les mots qui n'ont pas de vecteur associé
    word_vectors = [wordvec.wv[word] for word in words if word in wordvec.wv]  # Obtention des vecteurs 
    return word_vectors

# Appliquer la vectorisation à chaque message
data['Vector'] = data['v2'].apply(vectorize_message)
data.head()

# Convertir la liste de vecteurs en array numpy pour le padding
sequences = pad_sequences(data['Vector'].tolist(), padding='post', dtype='float32', value=0.0)

print("Shape of sequences: ", sequences.shape)

# Préparer les labels
labels = data['v1'].apply(lambda x: 1 if x == 'spam' else 0).values

print("Spam count: ", np.sum(labels))
print("Ham count: ", len(labels) - np.sum(labels))


## Entrainement du modèle

In [ ]:
def model():
    input = layers.Input(shape=(sequences.shape[1], sequences.shape[2]))
    x = layers.Masking(mask_value=0.0)(input) # Extrèmement important
    x = layers.BatchNormalization()(x)
    x = layers.LSTM(64)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(16, activation='leaky_relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(1, activation='sigmoid')(x)
    return models.Model(inputs=input, outputs=x, name='spam_detector')

model = model()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Callback d'early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Entraîner le modèle
history = model.fit(sequences, labels, batch_size=64, epochs=30, validation_split=0.2, callbacks=[early_stopping])

### Analyse de l'entrainement

In [ ]:

# Tracer l'historique de l'entraînement
plt.figure(figsize=(12, 6))

# Tracer la perte
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Perte d\'entraînement')
plt.plot(history.history['val_loss'], label='Perte de validation')
plt.title('Perte')
plt.xlabel('Épochs')
plt.ylabel('Perte')
plt.legend()

# Tracer l'exactitude
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Exactitude d\'entraînement')
plt.plot(history.history['val_accuracy'], label='Exactitude de validation')
plt.title('Exactitude')
plt.xlabel('Épochs')
plt.ylabel('Exactitude')
plt.legend()

plt.show()

## Evaluation du modèle

In [ ]:
# Prédire les labels pour les données d'entraînement
predictions = model.predict(sequences)

### Evaluation du modèle avec matrice de confusion

In [ ]:
Threshold = 0.5
Beta = 1.0

predicted_labels = (predictions > Threshold).astype(int).flatten()

# Calculer le recall
recall = recall_score(labels, predicted_labels)
print(f"Recall: {recall}")
# Calculer l'accuracy
accuracy = accuracy_score(labels, predicted_labels)
print(f"Accuracy: {accuracy}")
# Calculer le F-beta score
f_beta = fbeta_score(labels, predicted_labels, beta=Beta)
print(f"F-{Beta} Score: {f_beta}")

# Calculer la matrice de confusion
cm = confusion_matrix(labels, predicted_labels)

# Afficher la matrice de confusion
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['ham', 'spam'])
disp.plot(cmap=plt.cm.Blues)
plt.show()


### Evaluation du modèle avec une courbe ROC


In [ ]:

# Calculer les valeurs de la courbe ROC
fpr, tpr, thresholds = roc_curve(labels, predictions)

# Calculer l'AUC (Area Under Curve)
roc_auc = auc(fpr, tpr)

# Tracer la courbe ROC
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='Courbe ROC (aire = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.title('Courbe ROC')
plt.legend(loc="lower right")
plt.show()

### Test du modèle avec input utilisateur

In [ ]:
def predict_spam(message):
    # Nettoyer le message
    cleaned_message = clean_text(message)
    
    # Vectoriser le message
    vectorized_message = vectorize_message(cleaned_message)
    print("Length of vectorized message: ", len(vectorized_message))
    
    # Appliquer le padding
    padded_message = pad_sequences([vectorized_message], maxlen=sequences.shape[1], padding='post', truncating='post', dtype='float32', value=0.0)
    
    # Prédire avec le modèle
    prediction = model.predict(padded_message)
    
    # Retourner le résultat
    return 'spam' if prediction > Threshold else 'ham'



    # Exemple d'utilisation

    message1 = "Congratulations! You've won a free ticket to the Bahamas. Text WIN to 12345 to claim your prize."
message2 = "Hey, what are you doing later? Want to grab a cup of coffee?"
message3 = "Buy cheap viagra cialis online"
# 332 caractères
message4 = "Hey motherfucker, ahhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh"
result = predict_spam(message4)
print(f"The message is classified as: {result}")